In [1]:
import numpy as np
import time

def objective_function(solutions, cities):
    total_distance = 0
    for solution in solutions:
        distance = 0
        for i in range(len(solution) - 1):
            city1_index, city2_index = solution[i], solution[i+1]
            city1, city2 = cities[city1_index], cities[city2_index]
            distance += np.linalg.norm(np.array(city1) - np.array(city2))
        total_distance += distance
    return total_distance

def initialize_population(num_salesmen, exclusive_city_sets, shared_city_set, num_bees):
    food_sources = []
    for _ in range(num_bees):
        solutions = []
        for exclusive_cities in exclusive_city_sets:
            route = [1] + exclusive_cities + [1]  # Depot city is 1
            np.random.shuffle(route[1:-1])
            solutions.append(route)
        shared_cities = shared_city_set.copy()
        np.random.shuffle(shared_cities)
        for city in shared_cities:
            chosen_salesman = np.random.choice(range(num_salesmen))
            insert_pos = np.random.randint(1, len(solutions[chosen_salesman])-1)
            solutions[chosen_salesman].insert(insert_pos, city)
        food_sources.append(solutions)
    return food_sources

def generate_neighboring_solution(solution, cities, Pcp):
    solution = generate_neighboring_solution_intra(solution, cities, Pcp)
    solution = generate_neighboring_solution_inter(solution, cities, Pcp)
    return solution

def generate_neighboring_solution_intra(solution, cities, Pcp):
    new_solution = [route.copy() for route in solution]
    for route in new_solution:
        if np.random.rand() < Pcp:
            i, j = np.random.choice(range(1, len(route)-1), 2, replace=False)
            route[i], route[j] = route[j], route[i]
    return new_solution

def generate_neighboring_solution_inter(solution, cities, Pcp):
    new_solution = [route.copy() for route in solution]
    if np.random.rand() < Pcp:
        source_route_idx, target_route_idx = np.random.choice(range(len(solution)), 2, replace=False)
        source_route, target_route = new_solution[source_route_idx], new_solution[target_route_idx]
        if len(source_route) > 3:
            city_idx = np.random.randint(1, len(source_route)-1)
            city = source_route.pop(city_idx)
            insert_pos = np.random.randint(1, len(target_route)-1)
            target_route.insert(insert_pos, city)
    return new_solution

def select_food_sources(food_sources, cities, num_onlooker_bees):
    fitness_values = np.array([1 / (1 + objective_function(source, cities)) for source in food_sources])
    probabilities = fitness_values / fitness_values.sum()
    chosen_indices = np.random.choice(len(food_sources), size=num_onlooker_bees, p=probabilities)
    return [food_sources[idx] for idx in chosen_indices]

def ABC_CTSP_algorithm(num_salesmen, exclusive_city_sets, shared_city_set, num_bees, num_onlooker_bees, cities, runtime_seconds, limit_scout, Pmincp, Pmaxcp):
    start_time = time.time()
    food_sources = initialize_population(num_salesmen, exclusive_city_sets, shared_city_set, num_bees)
    best_solution = min(food_sources, key=lambda x: objective_function(x, cities))
    best_fitness = objective_function(best_solution, cities)

    while time.time() - start_time < runtime_seconds:
        Pcp = Pmincp + (Pmaxcp - Pmincp) * ((time.time() - start_time) / runtime_seconds)
        for i in range(len(food_sources)):
            new_source = generate_neighboring_solution(food_sources[i], cities, Pcp)
            if objective_function(new_source, cities) < objective_function(food_sources[i], cities):
                food_sources[i] = new_source

        onlooker_sources = select_food_sources(food_sources, cities, num_onlooker_bees)
        for source in onlooker_sources:
            new_source = generate_neighboring_solution(source, cities, Pcp)
            if objective_function(new_source, cities) < objective_function(source, cities):
                source[:] = new_source

        for i in range(len(food_sources)):
            if np.random.rand() < limit_scout / runtime_seconds:
                food_sources[i] = initialize_population(num_salesmen, exclusive_city_sets, shared_city_set, 1)[0]

        current_best = min(food_sources, key=lambda x: objective_function(x, cities))
        current_fitness = objective_function(current_best, cities)
        if current_fitness < best_fitness:
            best_solution, best_fitness = current_best, current_fitness

    return best_solution, best_fitness

def main():
    num_cities = int(input("Enter the number of cities: "))
    num_salesmen = int(input("Enter the number of salesmen: "))
    exclusive_city_sets = []
    for i in range(num_salesmen):
        cities = list(map(int, input(f"Enter exclusive cities for salesman {i+1}: ").strip().split()))
        exclusive_city_sets.append(cities)
    shared_city_set = list(map(int, input("Enter shared city set: ").strip().split()))
    runtime_seconds = float(input("Enter the number of seconds the algorithm should run: "))

    city_coordinates_text = """
    0 42 41
    1 32 22
    2 30 40
    3 25 32
    4 27 23
    5 31 32
    6 58 48
    7 49 49
    8 52 33
    9 52 41
    10 51 31
    11 40 30
    12 37 52
    13 42 50
    14 37 48
    15 38 46
    16 38 28
    17 48 28
    18 45 35
    19 43 25
    20 38 35
    """
    city_coordinates = {}
    for line in city_coordinates_text.strip().split('\n'):
        parts = line.split()
        city_num = int(parts[0]) + 1  # Correcting the city number
        coordinates = (int(parts[1]), int(parts[2]))
        city_coordinates[city_num] = coordinates

    # Fixed parameters
    num_bees = 100  # Number of employed bees
    num_onlooker_bees = 150  # Number of onlooker bees
    limit_scout = 125  # Limit for scout bees
    Pmincp = 0.1  # Minimum value of Pcp
    Pmaxcp = 0.7  # Maximum value of Pcp

    num_runs = 10
    best_solutions = []

    for _ in range(num_runs):
      best_solution, best_fitness = ABC_CTSP_algorithm(num_salesmen, exclusive_city_sets, shared_city_set, num_bees, num_onlooker_bees, city_coordinates, runtime_seconds, limit_scout, Pmincp, Pmaxcp)
      best_solutions.append(best_fitness)

    print("Results over 10 runs:")
    print("Best solution fitness:", min(best_solutions))
    print("Worst solution fitness:", max(best_solutions))
    print("Mean solution fitness:", np.mean(best_solutions))

if __name__ == "__main__":
    main()

Enter the number of cities: 21
Enter the number of salesmen: 2
Enter exclusive cities for salesman 1: 2 3 4 5 6
Enter exclusive cities for salesman 2: 7 8 9 10 11
Enter shared city set: 12 13 14 15 16 17 18 19 20 21
Enter the number of seconds the algorithm should run: 5
Results over 10 runs:
Best solution fitness: 228.91994152795317
Worst solution fitness: 255.65458896516185
Mean solution fitness: 244.67720270786495
